In [28]:
p_input = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

In [49]:
with open("3-1.txt") as f:
    p_input = f.read()

In [9]:
import re
import numpy as np

Ok, first idea is to have a list of position indicies for the mul instructions, do instructions, and don't instructions. I guess for each mul, we'd then check the equivelent position in the do and don't lists and move backwards, seeing if there was a match? If do was biggest or there isn't a match until the last mul, then the current mul is valid. otherwise, if the don't is bigger than the do or there is only a don't, it's not valid.

In [50]:
pattern = r"mul\((\d+),(\d+)\)"
matches = re.finditer(pattern, p_input)
muls = np.array([match.start() for match in matches])
vals = [int(int(x) * int(y)) for x, y in (match.groups() for match in re.finditer(pattern, p_input))]
print(vals)

pattern = r"do()"
matches = re.finditer(pattern, p_input)
dos = np.array([match.start() for match in matches])

pattern = r"don't()"
matches = re.finditer(pattern, p_input)
donts = np.array([match.start() for match in matches])

[765336, 374300, 3740, 328563, 304920, 603693, 92130, 262213, 51125, 14430, 19800, 355509, 121905, 590240, 16378, 583583, 52884, 177944, 401120, 488992, 266686, 445041, 622665, 73780, 19864, 382200, 329478, 387604, 71808, 51972, 188160, 165984, 214465, 226904, 362604, 168251, 14200, 190032, 682958, 602572, 669536, 25246, 146248, 128526, 29011, 86296, 90628, 122846, 63063, 40183, 33748, 416010, 226284, 10260, 346577, 336753, 155011, 82488, 143856, 338247, 491712, 546832, 104838, 294808, 226746, 55822, 390128, 69498, 48240, 219924, 106008, 319440, 31970, 350637, 7389, 208581, 94320, 17940, 210392, 512544, 578375, 280098, 153192, 146248, 1214, 16900, 96792, 233934, 68076, 316448, 244020, 403848, 283632, 470752, 2940, 192, 621810, 245385, 214434, 41912, 48655, 33374, 249168, 290043, 127120, 196098, 2121, 376855, 452304, 110600, 126820, 874248, 44286, 709932, 109135, 319606, 96333, 314018, 450826, 55350, 273498, 19680, 181720, 2136, 25298, 488064, 459108, 48636, 140400, 26068, 4620, 18256, 

In [51]:
print(muls)
print(dos)
print(donts)

[    2    17    44    69   109   149   173   189   213   245   288   309
   337   365   380   391   415   448   477   492   505   518   531   563
   595   607   643   656   700   718   748   768   805   826   846   898
   935   952   982   994  1006  1031  1043  1076  1100  1121  1157  1183
  1198  1246  1264  1277  1334  1370  1387  1413  1427  1448  1464  1495
  1549  1612  1641  1672  1698  1728  1759  1780  1804  1823  1835  1858
  1884  1907  1930  1940  1963  1981  2019  2049  2083  2120  2195  2239
  2265  2277  2297  2336  2368  2393  2420  2433  2510  2542  2555  2593
  2628  2684  2715  2747  2782  2819  2830  2843  2856  2874  2902  2938
  3014  3045  3085  3109  3125  3156  3207  3232  3265  3301  3334  3375
  3396  3424  3446  3464  3496  3534  3569  3584  3597  3624  3635  3654
  3676  3688  3705  3737  3789  3841  3858  3887  3947  3973  4017  4061
  4086  4117  4144  4161  4186  4198  4241  4281  4310  4337  4381  4408
  4431  4480  4505  4518  4536  4570  4593  4625  4

In [52]:
last_mul_pos = 0
for mul_pos in muls:
    # check the do/don't lists
    relevant_dos = dos[(dos > last_mul_pos) & (dos < mul_pos)]
    relevant_dont = donts[(donts > last_mul_pos) & (donts < mul_pos)]

    # get the max or 0
    max_do_i = max(relevant_dos, default=-1)
    max_dont_i = max(relevant_dont, default=-1)

    # run a comparison between the four possible conditions
    if max_dont_i == -1 or max_do_i > max_dont_i:
        print("Valid")
    else:
        print("Invalid")

    last_mul_pos = mul_pos

Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Invalid
Invalid
Invalid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Valid
Vali

Ah, I misunderstood. Do/Don't applies to _all_ muls until undone by one or the other, and there's essentially a do in the beginning.

This should simplify things? We have those initial lists which almost mark pivot points. For each mul, we should check both lists to see if they have a value below ours, and the larger value is if we count or not. If neither has a value, it counts (default do to -1, don't to -2?)

To find the index, we could split the list using an index we find from a search (O(n)), or create a new list up to the point where it's lower (also O(n)). Index sounds a bit better to me.

In [53]:
last_mul_pos = 0
total = 0
for mul_i, mul_pos in enumerate(muls):
    # get the max or 0
    max_do_i = np.max(dos[dos <= mul_pos], initial=-1)
    max_dont_i = np.max(donts[donts <= mul_pos], initial=-2)

    # run a comparison between the four possible conditions
    if max_do_i > max_dont_i:
        total += vals[mul_i]

    last_mul_pos = mul_pos
print(total)

99532691


Ok this works but it is _really_ ugly. This is just simple state changes, running a loop where the state changes at the right time should be fine. The current approach does lookups for what the state should be each time.

Claude gave me the following:

In [55]:
import re

def process_instructions(text):
    # Track the enable/disable state as we process
    enabled = True
    total = 0
    
    # First, let's find all our control instructions and multiplications
    # We'll store them as tuples of (position, type, value)
    instructions = []
    
    # Find all mul instructions with their values
    for match in re.finditer(r'mul\((\d+),(\d+)\)', text):
        pos = match.start()
        x, y = map(int, match.groups())
        instructions.append((pos, 'mul', x * y))
    
    # Find all control instructions
    for match in re.finditer(r'do\(\)', text):
        instructions.append((match.start(), 'do', None))
    
    for match in re.finditer(r"don't\(\)", text):
        instructions.append((match.start(), 'dont', None))
    
    # Sort by position to process in order
    instructions.sort()
    
    # Process instructions in order
    for pos, type, value in instructions:
        if type == 'do':
            enabled = True
        elif type == 'dont':
            enabled = False
        elif type == 'mul' and enabled:
            total += value
    
    return total

# Test with the example
test_input = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)do()?mul(8,5))"
print(f"Test result: {process_instructions(test_input)}")

Test result: 48
